---
title: Geostationary Satellite Observations of Extreme and Transient Methane Emissions from Oil and Gas Infrastructure
description: A sample of methane plumes from point sources observed since 2019 by the U.S. Geostationary Operational Environmental Satellites (GOES) over North and South America.
author: Siddharth Chaudhary
date: 15 November 2024
execute:
   freeze: true
---